In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# CK+ Results

In [2]:
%cd ~/Desktop/research_project_repo/research_project/experiments/ckplus/2021-04-26-04-51/

/Users/samuilstoychev/Desktop/research_project_repo/research_project/experiments/ckplus/2021-04-26-04-51


In [3]:
!ls

gr_112    grd_112   lgr_112   lgrd_112  lr_112    nr_112
gr_1229   grd_1229  lgr_1229  lgrd_1229 lr_1229   nr_1229
gr_8032   grd_8032  lgr_8032  lgrd_8032 lr_8032   nr_8032


In [4]:
logs = !ls

In [17]:
def extract_metrics_gpu(filename): 
    f = open(filename, 'r')
    lines = f.readlines() 
    ram_init = None
    ram_peak = None
    cpu_init = None 
    cpu_end = None 
    for line in lines: 
        if "=> Average precision" in line: 
            accuracy = float(line.split(" ")[-1])
        if "=> Total training time" in line: 
            training_time = float(line.split(" ")[-2])
        if "RAM BEFORE TRAINING:" in line: 
            ram_init = float(line.split(" ")[-1])
        if "PEAK TRAINING RAM:" in line: 
            ram_peak = float(line.split(" ")[-1])
        if "CPU BEFORE TRAINING:" in line: 
            cpu_init = eval(line.split(":")[-1])
        if "CPU BEFORE EVALUATION:" in line: 
            cpu_end = eval(line.split(":")[-1])
        if "GPU BEFORE EVALUATION:" in line: 
            gpu_u, gpu_mem = eval(line.split(":")[-1])
        
    cpu_u, cpu_k = (cpu_end[0] - cpu_init[0], cpu_end[1] - cpu_init[1])
    return (accuracy * 100, training_time, (ram_peak-ram_init) * 1024, cpu_u + cpu_k, gpu_u, gpu_mem)

In [18]:
def calculate_metrics_gpu(files): 
    metrics = dict()
    
    for file in files: 
        method, seed = file.split("_") 
        
        if method in metrics: 
            metrics[method].append(extract_metrics_gpu(file))
        else: 
            metrics[method] = [extract_metrics_gpu(file)] 
    
    return metrics

In [19]:
def get_means(metrics): 
    return {method: [sum(x)/len(x) for x in list(zip(*metrics[method]))] for method in metrics}

In [20]:
def get_sds(metrics): 
    return {method: [np.std(x) for x in list(zip(*metrics[method]))] for method in metrics}

In [21]:
ckplus_metrics = calculate_metrics_gpu(logs)
ckplus_means = get_means(ckplus_metrics)
ckplus_sds = get_sds(ckplus_metrics)

In [26]:
df = pd.DataFrame(ckplus_means, index=['Accuracy (%)', 
                                 'Training time (s)', 
                                 'RAM Usage (MB)', 
                                 'CPU Usage (s)', 
                                 'GPU Usage (%)', 
                                 'GPU Memory (MB)'])
df = df.filter(["nr", "lr", "gr", "lgr", "grd", "lgrd"])

In [27]:
df

,nr,lr,gr,lgr,grd,lgrd
Accuracy (%),82.723333,79.783333,75.080000,79.536667,72.946667,81.170000
Training time (s),703.300000,625.966667,850.266667,678.900000,855.966667,673.066667
RAM Usage (MB),15.488281,14.024740,17.139323,15.912760,17.074219,15.601562
CPU Usage (s),755.233333,676.026667,902.680000,727.663333,909.466667,724.180000
GPU Usage (%),24.021596,16.000256,34.772868,16.282441,31.767442,16.350161
GPU Memory (MB),2095.333333,114.000000,2710.000000,14.000000,2710.000000,14.000000
